In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
df = pd.read_csv("../../input/train.csv")
qualitative = list(df.columns[df.dtypes == "object"])
#qualitative += [x for x in df.select_dtypes(include=['float64',"int64"]).columns if len(df[x].unique()) <= 10]
#quantitative = [x for x in df.select_dtypes(include=['float64',"int64"]).columns if len(df[x].unique()) > 10]
quantitative = df.select_dtypes(include=['float64',"int64"]).columns[:-1]
df[qualitative] = df[qualitative].fillna("None")
df[quantitative] = df[quantitative].fillna(0)


In [55]:
qualDf = pd.get_dummies(df[qualitative])
cats = qualDf.columns
X = pd.concat([df[quantitative],qualDf],axis=1)
y = df.SalePrice

In [56]:
tf = pd.read_csv("../../input/test.csv")
onehot_frame = []
for num in range(len(tf)):
    row = tf[qualitative].iloc[num]
    values = [x + "_"+ str(y) for x, y in row.items()]
    onehot_frame.append([1 if x in values else 0 for x in cats])
qualTf = pd.DataFrame(onehot_frame, columns=cats)

Xtest = pd.concat([tf[quantitative],qualTf],axis=1)

In [57]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [58]:
import xgboost as xg 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.metrics import r2_score
# Instantiation 
xgb_r = xg.XGBRegressor(n_estimators = 100, seed = 123, max_depth=3, learning_rate=.08, colsample_bytree=1)
#col sample is how many they choose for tree like random forest
  
# Fitting the model 
xgb_r.fit(x_train, y_train) 
  
# Predict the model 
predictions = xgb_r.predict(x_test) 

# RMSE Computation 
rmse = np.sqrt(MSE(y_test, predictions)) 
print("RMSE : % f" %(rmse)) 

print("Test score:")
r2_score(y_test, predictions)

RMSE :  30455.870195
Test score:


0.8790716321238956

In [62]:
Xtest[list(X.columns)]

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,0,0,0,1,0,0,0,0,1,0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,0,0,0,1,0,0,0,0,1,0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,0,0,0,1,0,0,0,0,1,0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,0,0,0,1,0,0,0,0,1,0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,...,0,0,0,1,0,0,0,0,1,0
1455,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,...,0,0,0,1,1,0,0,0,0,0
1456,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,...,0,0,0,1,1,0,0,0,0,0
1457,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,...,0,0,0,1,0,0,0,0,1,0


In [63]:
predictions = xgb_r.predict(Xtest)

In [65]:
pd.DataFrame({'SalePrice':predictions},tf.Id).to_csv("../../output/XGBoost_oneHot_submission.csv")